<a href="https://colab.research.google.com/github/jidan-fikri/purino-workshop/blob/master/Purino_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
!git clone https://github.com/jidan-fikri/purino-workshop.git

Cloning into 'purino-workshop'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 23 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [3]:
tf.__version__

'2.8.2'

In [14]:
voice_data = pd.read_csv('/content/purino-workshop/dataset/voice.csv')
iris_data = pd.read_csv('/content/purino-workshop/dataset/Iris.csv')

voice_data['label'].replace(['male', 'female'], [0, 1], inplace=True)
iris_data['Species'].replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], [0, 1, 2], inplace=True)

In [15]:
voice_data

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,0
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,0
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,0
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,0
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0.131884,0.084734,0.153707,0.049285,0.201144,0.151859,1.762129,6.630383,0.962934,0.763182,...,0.131884,0.182790,0.083770,0.262295,0.832899,0.007812,4.210938,4.203125,0.161929,1
3164,0.116221,0.089221,0.076758,0.042718,0.204911,0.162193,0.693730,2.503954,0.960716,0.709570,...,0.116221,0.188980,0.034409,0.275862,0.909856,0.039062,3.679688,3.640625,0.277897,1
3165,0.142056,0.095798,0.183731,0.033424,0.224360,0.190936,1.876502,6.604509,0.946854,0.654196,...,0.142056,0.209918,0.039506,0.275862,0.494271,0.007812,2.937500,2.929688,0.194759,1
3166,0.143659,0.090628,0.184976,0.043508,0.219943,0.176435,1.591065,5.388298,0.950436,0.675470,...,0.143659,0.172375,0.034483,0.250000,0.791360,0.007812,3.593750,3.585938,0.311002,1


In [16]:
iris_data

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,2
146,147,6.3,2.5,5.0,1.9,2
147,148,6.5,3.0,5.2,2.0,2
148,149,6.2,3.4,5.4,2.3,2


# Voice Model

In [17]:
#bagi data independen dan dependen variable
xv = voice_data.iloc[:,0:20] #independen variable
yv = voice_data.iloc[:,20] #dependen variable


In [18]:
from sklearn.model_selection import train_test_split

Xv_train, Xv_test, yv_train, yv_test = train_test_split(xv, yv, test_size=0.2)

In [24]:
from tensorflow import keras
from keras.layers import Dense, BatchNormalization
from keras.models import Sequential
from keras.callbacks import CSVLogger

model = Sequential()
model.add(Dense(13, activation='relu', input_shape=(20,))) #input layer
model.add(BatchNormalization())
model.add(Dense(10, activation='relu')) #hidden layer
model.add(BatchNormalization())
model.add(Dense(5, activation='relu')) #hidden layer
model.add(BatchNormalization())
model.add(Dense(1)) #output layer

model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
csv_logger = CSVLogger('/content/log_voice.csv')
model.fit(Xv_train, yv_train, validation_split=0.2, epochs=150, callbacks=[csv_logger])

Epoch 1/150
64/64 [==============================] - 1s 6ms/step - loss: 1.0094 - accuracy: 0.5555 - val_loss: 1.4437 - val_accuracy: 0.5759
Epoch 2/150
64/64 [==============================] - 0s 3ms/step - loss: 0.3977 - accuracy: 0.5678 - val_loss: 0.5897 - val_accuracy: 0.5207
Epoch 3/150
64/64 [==============================] - 0s 3ms/step - loss: 0.3259 - accuracy: 0.5688 - val_loss: 0.3037 - val_accuracy: 0.5286
Epoch 4/150
64/64 [==============================] - 0s 4ms/step - loss: 0.2794 - accuracy: 0.6038 - val_loss: 0.2812 - val_accuracy: 0.5404
Epoch 5/150
64/64 [==============================] - 0s 3ms/step - loss: 0.2613 - accuracy: 0.5999 - val_loss: 0.2481 - val_accuracy: 0.5562
Epoch 6/150
64/64 [==============================] - 0s 3ms/step - loss: 0.2407 - accuracy: 0.6147 - val_loss: 0.2423 - val_accuracy: 0.5759
Epoch 7/150
64/64 [==============================] - 0s 3ms/step - loss: 0.2382 - accuracy: 0.6339 - val_loss: 0.2293 - val_accuracy: 0.6529
Epoch 8/150
6

# Iris model

In [20]:
#bagi data independen dan dependen variable
xi = iris_data.iloc[:,1:5] #independen variable
yi = iris_data.iloc[:,5] #dependen variable

In [21]:
from sklearn.model_selection import train_test_split

Xi_train, Xi_test, yi_train, yi_test = train_test_split(xi, yi, test_size=0.2)

In [32]:
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import CSVLogger

model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(4,))) #input layer
model.add(Dense(10, activation='relu')) #hidden layer
model.add(Dense(5, activation='relu')) #hidden layer
model.add(Dense(3, activation = 'softmax')) #output layer

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
csv_logger = CSVLogger('/content/log_iris.csv')
model.fit(Xi_train, yi_train, validation_split=0.2, epochs=150, callbacks=[csv_logger])

Epoch 1/150
3/3 [==============================] - 1s 84ms/step - loss: 1.1101 - accuracy: 0.3646 - val_loss: 1.1703 - val_accuracy: 0.2500
Epoch 2/150
3/3 [==============================] - 0s 13ms/step - loss: 1.0834 - accuracy: 0.3646 - val_loss: 1.1398 - val_accuracy: 0.2500
Epoch 3/150
3/3 [==============================] - 0s 27ms/step - loss: 1.0602 - accuracy: 0.3646 - val_loss: 1.1156 - val_accuracy: 0.2500
Epoch 4/150
3/3 [==============================] - 0s 20ms/step - loss: 1.0376 - accuracy: 0.3646 - val_loss: 1.0966 - val_accuracy: 0.2500
Epoch 5/150
3/3 [==============================] - 0s 19ms/step - loss: 1.0203 - accuracy: 0.3646 - val_loss: 1.0783 - val_accuracy: 0.2500
Epoch 6/150
3/3 [==============================] - 0s 41ms/step - loss: 1.0040 - accuracy: 0.3646 - val_loss: 1.0634 - val_accuracy: 0.2500
Epoch 7/150
3/3 [==============================] - 0s 42ms/step - loss: 0.9892 - accuracy: 0.3646 - val_loss: 1.0488 - val_accuracy: 0.2500
Epoch 8/150
3/3 [===

# MNIST Handwritting